In [1]:
import os
import numpy as np
import requests
import re
import copy
import networkx as nx

# Get Input

In [2]:
sessionId = os.environ["ADVENT_OF_CODE_SESSION_ID"]

In [3]:
r = requests.get("https://adventofcode.com/2018/day/19/input", cookies={"session": sessionId}, verify=False)

/Users/cmartin/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
input = r.content

In [5]:
lines = [line.decode('utf-8') for line in input.splitlines()]

In [7]:
lines

['#ip 2',
 'addi 2 16 2',
 'seti 1 2 4',
 'seti 1 8 1',
 'mulr 4 1 5',
 'eqrr 5 3 5',
 'addr 5 2 2',
 'addi 2 1 2',
 'addr 4 0 0',
 'addi 1 1 1',
 'gtrr 1 3 5',
 'addr 2 5 2',
 'seti 2 6 2',
 'addi 4 1 4',
 'gtrr 4 3 5',
 'addr 5 2 2',
 'seti 1 2 2',
 'mulr 2 2 2',
 'addi 3 2 3',
 'mulr 3 3 3',
 'mulr 2 3 3',
 'muli 3 11 3',
 'addi 5 2 5',
 'mulr 5 2 5',
 'addi 5 8 5',
 'addr 3 5 3',
 'addr 2 0 2',
 'seti 0 4 2',
 'setr 2 5 5',
 'mulr 5 2 5',
 'addr 2 5 5',
 'mulr 2 5 5',
 'muli 5 14 5',
 'mulr 5 2 5',
 'addr 3 5 3',
 'seti 0 8 0',
 'seti 0 5 2']

In [9]:
pointer = re.findall("\d+",lines[0])[0]
pointer

'2'

In [11]:
instructions = []
for line in lines[1:]:
    op = line[:4]
    args = [int(n) for n in re.findall("\d+",line)]
    instructions.append((op,args))

In [45]:
ops[0]

{'before': [1, 1, 2, 0], 'op': [8, 1, 0, 3], 'after': [1, 1, 2, 1]}

# Setup

In [14]:
class Instructions:
    @staticmethod
    def addr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] + registers[B]
        return registers

    @staticmethod
    def addi(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] + B
        return registers

    @staticmethod
    def mulr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] * registers[B]
        return registers

    @staticmethod
    def muli(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] * B
        return registers

    @staticmethod
    def banr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] & registers[B]
        return registers

    @staticmethod
    def bani(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] & B
        return registers

    @staticmethod
    def borr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] | registers[B]
        return registers

    @staticmethod
    def bori(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A] | B
        return registers

    @staticmethod
    def setr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = registers[A]
        return registers

    @staticmethod
    def seti(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = A
        return registers

    @staticmethod
    def gtir(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = 1 if A > registers[B] else 0
        return registers

    @staticmethod
    def gtri(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = 1 if registers[A] > B else 0
        return registers

    @staticmethod
    def gtrr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = 1 if registers[A] > registers[B] else 0
        return registers

    @staticmethod
    def eqir(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = 1 if A ==registers[B] else 0
        return registers

    @staticmethod
    def eqri(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = 1 if registers[A] == B else 0
        return registers

    @staticmethod
    def eqrr(registers_in, A, B, C):
        registers = copy.copy(registers_in)
        registers[C] = 1 if registers[A] == registers[B] else 0
        return registers
    
    @staticmethod
    def instrctions():
        return [addr,
                 addi,
                 mulr, 
                 muli,
                 banr, 
                 bani, 
                 borr, 
                 bori, 
                 setr, 
                 seti, 
                 gtir, 
                 gtri, 
                 gtrr, 
                 eqir, 
                 eqri, 
                 eqrr]

# Test Case

In [15]:
test_input = """#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5"""

In [21]:
instructions = []
for line in test_input.splitlines()[1:]:
    op = getattr(Instructions,line[:4])
    args = [int(n) for n in re.findall("\d+",line)]
    
    instructions.append((op,args))

In [22]:
instructions

[(<function __main__.Instructions.seti(registers_in, A, B, C)>, [5, 0, 1]),
 (<function __main__.Instructions.seti(registers_in, A, B, C)>, [6, 0, 2]),
 (<function __main__.Instructions.addi(registers_in, A, B, C)>, [0, 1, 0]),
 (<function __main__.Instructions.addr(registers_in, A, B, C)>, [1, 2, 3]),
 (<function __main__.Instructions.setr(registers_in, A, B, C)>, [1, 0, 0]),
 (<function __main__.Instructions.seti(registers_in, A, B, C)>, [8, 0, 4]),
 (<function __main__.Instructions.seti(registers_in, A, B, C)>, [9, 0, 5])]

In [32]:
ip_register = 0
register = [0]*6
iters = 0
while True:
    iters += 1
    ip = register[ip_register]
    if ip < 0 or ip >= len(instructions):
        break
    prev_register = register[:]
    op, args = instructions[ip]
    register = op(register,*args)
    print("ip={} {} {} {} {}".format(ip, prev_register, op.__name__, " ".join([str(a) for a in args]), register))
    register[ip_register] += 1
    
    

    

ip=0 [0, 0, 0, 0, 0, 0] seti 5 0 1 [0, 5, 0, 0, 0, 0]
ip=1 [1, 5, 0, 0, 0, 0] seti 6 0 2 [1, 5, 6, 0, 0, 0]
ip=2 [2, 5, 6, 0, 0, 0] addi 0 1 0 [3, 5, 6, 0, 0, 0]
ip=4 [4, 5, 6, 0, 0, 0] setr 1 0 0 [5, 5, 6, 0, 0, 0]
ip=6 [6, 5, 6, 0, 0, 0] seti 9 0 5 [6, 5, 6, 0, 0, 9]


# Part 1

In [37]:
ip_register = int(re.findall("\d+",lines[0])[0])
ip_register

2

In [38]:
instructions = []
for line in lines[1:]:
    op = getattr(Instructions,line[:4])
    args = [int(n) for n in re.findall("\d+",line)]
    
    instructions.append((op,args))

In [43]:

register = [0]*6
iters = 0
while True and iters < 100000000:
    iters+=1
    ip = register[ip_register]
    if ip < 0 or ip >= len(instructions):
        break
    prev_register = register[:]
    op, args = instructions[ip]
    register = op(register,*args)
    #print("ip={} {} {} {} {}".format(ip, prev_register, op.__name__, " ".join([str(a) for a in args]), register))
    register[ip_register] += 1


In [44]:
iters

6311917

In [46]:
register

[2280, 889, 257, 888, 889, 1]

# Part 2

In [91]:

register = [1,0,0,0,0,0]

iters = 0
while True and iters < 1000:
    iters+=1
    ip = register[ip_register]
    if ip < 0 or ip >= len(instructions):
        break
    prev_register = register[:]
    op, args = instructions[ip]
    register = op(register,*args)
    print("ip={} {} {} {} {}".format(ip, prev_register, op.__name__, " ".join([str(a) for a in args]), register))
    register[ip_register] += 1

ip=0 [1, 0, 0, 0, 0, 0] addi 2 16 2 [1, 0, 16, 0, 0, 0]
ip=17 [1, 0, 17, 0, 0, 0] addi 3 2 3 [1, 0, 17, 2, 0, 0]
ip=18 [1, 0, 18, 2, 0, 0] mulr 3 3 3 [1, 0, 18, 4, 0, 0]
ip=19 [1, 0, 19, 4, 0, 0] mulr 2 3 3 [1, 0, 19, 76, 0, 0]
ip=20 [1, 0, 20, 76, 0, 0] muli 3 11 3 [1, 0, 20, 836, 0, 0]
ip=21 [1, 0, 21, 836, 0, 0] addi 5 2 5 [1, 0, 21, 836, 0, 2]
ip=22 [1, 0, 22, 836, 0, 2] mulr 5 2 5 [1, 0, 22, 836, 0, 44]
ip=23 [1, 0, 23, 836, 0, 44] addi 5 8 5 [1, 0, 23, 836, 0, 52]
ip=24 [1, 0, 24, 836, 0, 52] addr 3 5 3 [1, 0, 24, 888, 0, 52]
ip=25 [1, 0, 25, 888, 0, 52] addr 2 0 2 [1, 0, 26, 888, 0, 52]
ip=27 [1, 0, 27, 888, 0, 52] setr 2 5 5 [1, 0, 27, 888, 0, 27]
ip=28 [1, 0, 28, 888, 0, 27] mulr 5 2 5 [1, 0, 28, 888, 0, 756]
ip=29 [1, 0, 29, 888, 0, 756] addr 2 5 5 [1, 0, 29, 888, 0, 785]
ip=30 [1, 0, 30, 888, 0, 785] mulr 2 5 5 [1, 0, 30, 888, 0, 23550]
ip=31 [1, 0, 31, 888, 0, 23550] muli 5 14 5 [1, 0, 31, 888, 0, 329700]
ip=32 [1, 0, 32, 888, 0, 329700] mulr 5 2 5 [1, 0, 32, 888, 0, 105504

In [ ]:
iters

In [ ]:
register

In [124]:
register = [1, 10551287, 3, 10551288, 10551287, 0]
iters = 0
while True and iters < 1000:
    iters+=1
    ip = register[ip_register]
    if ip < 0 or ip >= len(instructions):
        break
    prev_register = register[:]
    op, args = instructions[ip]
    register = op(register,*args)
    print("ip={} {} {} {} {}".format(ip, prev_register, op.__name__, " ".join([str(a) for a in args]), register))
    register[ip_register] += 1
iters

ip=3 [1, 10551287, 3, 10551288, 10551287, 0] mulr 4 1 5 [1, 10551287, 3, 10551288, 10551287, 111329657356369]
ip=4 [1, 10551287, 4, 10551288, 10551287, 111329657356369] eqrr 5 3 5 [1, 10551287, 4, 10551288, 10551287, 0]
ip=5 [1, 10551287, 5, 10551288, 10551287, 0] addr 5 2 2 [1, 10551287, 5, 10551288, 10551287, 0]
ip=6 [1, 10551287, 6, 10551288, 10551287, 0] addi 2 1 2 [1, 10551287, 7, 10551288, 10551287, 0]
ip=8 [1, 10551287, 8, 10551288, 10551287, 0] addi 1 1 1 [1, 10551288, 8, 10551288, 10551287, 0]
ip=9 [1, 10551288, 9, 10551288, 10551287, 0] gtrr 1 3 5 [1, 10551288, 9, 10551288, 10551287, 0]
ip=10 [1, 10551288, 10, 10551288, 10551287, 0] addr 2 5 2 [1, 10551288, 10, 10551288, 10551287, 0]
ip=11 [1, 10551288, 11, 10551288, 10551287, 0] seti 2 6 2 [1, 10551288, 2, 10551288, 10551287, 0]
ip=3 [1, 10551288, 3, 10551288, 10551287, 0] mulr 4 1 5 [1, 10551288, 3, 10551288, 10551287, 111329667907656]
ip=4 [1, 10551288, 4, 10551288, 10551287, 111329667907656] eqrr 5 3 5 [1, 10551288, 4, 10

1000

In [126]:
register = [10551289, 10551287, 6, 10551288, 10551288, 0]
iters = 0
while True and iters < 1000:
    iters+=1
    ip = register[ip_register]
    if ip < 0 or ip >= len(instructions):
        break
    prev_register = register[:]
    op, args = instructions[ip]
    register = op(register,*args)
    print("ip={} {} {} {} {}".format(ip, prev_register, op.__name__, " ".join([str(a) for a in args]), register))
    register[ip_register] += 1

ip=6 [10551289, 10551287, 6, 10551288, 10551288, 0] addi 2 1 2 [10551289, 10551287, 7, 10551288, 10551288, 0]
ip=8 [10551289, 10551287, 8, 10551288, 10551288, 0] addi 1 1 1 [10551289, 10551288, 8, 10551288, 10551288, 0]
ip=9 [10551289, 10551288, 9, 10551288, 10551288, 0] gtrr 1 3 5 [10551289, 10551288, 9, 10551288, 10551288, 0]
ip=10 [10551289, 10551288, 10, 10551288, 10551288, 0] addr 2 5 2 [10551289, 10551288, 10, 10551288, 10551288, 0]
ip=11 [10551289, 10551288, 11, 10551288, 10551288, 0] seti 2 6 2 [10551289, 10551288, 2, 10551288, 10551288, 0]
ip=3 [10551289, 10551288, 3, 10551288, 10551288, 0] mulr 4 1 5 [10551289, 10551288, 3, 10551288, 10551288, 111329678458944]
ip=4 [10551289, 10551288, 4, 10551288, 10551288, 111329678458944] eqrr 5 3 5 [10551289, 10551288, 4, 10551288, 10551288, 0]
ip=5 [10551289, 10551288, 5, 10551288, 10551288, 0] addr 5 2 2 [10551289, 10551288, 5, 10551288, 10551288, 0]
ip=6 [10551289, 10551288, 6, 10551288, 10551288, 0] addi 2 1 2 [10551289, 10551288, 7, 

In [114]:
iters

20

In [128]:
import re
import collections
input_lines = lines[:]
a,b = map(int, [re.findall('\d+', input_lines[i])[1] for i in [22, 24]])
number_to_factorize = 10551236 + a * 22 + b

factors = collections.defaultdict(lambda: 0)
possible_prime_divisor = 2
while possible_prime_divisor ** 2 <= number_to_factorize:
  while number_to_factorize % possible_prime_divisor == 0:
    number_to_factorize /= possible_prime_divisor
    factors[possible_prime_divisor] += 1 
  possible_prime_divisor += 1
if number_to_factorize > 1:
  factors[number_to_factorize] += 1

sum_of_divisors = 1
for prime_factor in factors:
  sum_of_divisors *= (prime_factor ** (factors[prime_factor] + 1) - 1) / (prime_factor - 1)

print(sum_of_divisors)

30481920.0


In [129]:
number_to_factorize

2351.0

In [130]:
factors

defaultdict(<function __main__.<lambda>()>,
            {2: 3, 3: 1, 11: 1, 17: 1, 2351.0: 1})

In [131]:
2351**2

5527201

In [132]:
10551236 + a * 22 + b

10551288

In [135]:
2**3*3*11*17*2351

10551288